In [61]:
import pandas as pd

# Merge the datasets
data = pd.read_csv(r'C:\Users\DimitrideBoer\OneDrive - Emixa\Documents\XRAYxEmixaHackathon\data\RecommenderSystem_withoutcompany300520241508.csv')

C:\Users\DimitrideBoer\AppData\Local\Temp\ipykernel_36940\917879260.py:4: DtypeWarning: Columns (4,5,7,10,12,27,30,40,49,56,59,61,69,72,74,76,77,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\DimitrideBoer\OneDrive - Emixa\Documents\XRAYxEmixaHackathon\data\RecommenderSystem_withoutcompany300520241508.csv')


In [72]:
# Function to convert columns
def convert_columns(df):
    for column in df.columns:
        if column.startswith('is') or column.startswith('has'):
            df[column] = df[column].astype(bool)
        else:
            if pd.api.types.is_numeric_dtype(df[column]):
                df[column] = pd.to_numeric(df[column], errors='coerce')
            else:
                df[column] = df[column].astype(str)
    return df

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [62]:
# Automatically identify column types
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features = [ 'hasairco',
 'hascosts',
 'cardistancevalue']
numerical_features

['hasairco', 'hascosts', 'cardistancevalue']

In [63]:
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
categorical_features = ['tirerating',
 'electricalrating',
 'fueltype',
 'interiorcolor']

In [48]:
# Convert specific columns to string dtype
#data[['tirerating', 'electricalrating', 'fueltype', 'interiorcolor']] = data[['tirerating', 'electricalrating', 'fueltype', 'interiorcolor']].astype(str)

In [64]:
# Preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='constant', fill_value='most frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

preprocessor.fit_transform(data)

<373032x8215 sparse matrix of type '<class 'numpy.float64'>'
	with 2611224 stored elements in Compressed Sparse Row format>

In [65]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [66]:
model = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KMeans(n_clusters=8, random_state=42))])

In [67]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['hasairco', 'hascosts',
                                                   'cardistancevalue']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['tirerating',
                                                   'electricalrating',
                                                   'fueltype',
                                                   'interiorcolor'])])),
                ('classifier', KMeans(random_state=42))])

Clustering

In [68]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# # Determine the optimal number of clusters using the elbow method
# inertia = []
# K = range(1, 11)

# for k in K:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(car_features_preprocessed)
#     inertia.append(kmeans.inertia_)

# # Plot the elbow curve
# plt.figure(figsize=(8, 6))
# plt.plot(K, inertia, 'bx-')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Optimal Number of Clusters')
# plt.show()

# From the elbow plot, let's assume the optimal number of clusters is 3
optimal_clusters = 8
#kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
data['cluster'] = model.fit_predict(data[numerical_features + categorical_features])


c:\Users\DimitrideBoer\OneDrive - Emixa\Documents\XRAYxEmixaHackathon\venv\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [71]:
data[numerical_features + categorical_features + ['cluster']]

,hasairco,hascosts,cardistancevalue,tirerating,electricalrating,fueltype,interiorcolor,cluster
0,0,0,30000.0,Goed,Goed,Diesel,NaN,0
1,0,0,92000.0,Goed,Goed,Diesel,NaN,0
2,0,0,37545.0,Goed,Goed,Benzine,Zwart stof,0
3,0,0,192304.0,Goed,Goed,Diesel,Beige leder,0
4,0,0,34000.0,Goed,Goed,Diesel,Antraciet,0
...,...,...,...,...,...,...,...,...
373027,1,1,149886.0,NaN,NaN,Hybride benzine,Leder Zwart,1
373028,1,1,108913.0,NaN,NaN,Benzine,Stof Donkergrijs,7
373029,1,1,32000.0,NaN,NaN,Elektrisch,Leder Beige,1
373030,1,0,248479.0,NaN,NaN,Benzine,Stof Donkergrijs,5


Save model to ONNX

In [4]:
#! pip install onnx==1.13.1

In [5]:
#import onnx

In [6]:
#from onnx.defs import onnx_opset_version

In [7]:
#onnx_opset_version()

In [8]:
#onnx.__version__

In [15]:
car_features_preprocessed.shape

(373032, 10397)

In [64]:
len(data[numerical_features].columns)

43

In [69]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType

# Define the initial type for the input
initial_type = [('float_input', FloatTensorType([None, data[numerical_features].shape[1]]))]

initial_type = [('hasairco', FloatTensorType([1, 1])),
                ('hascosts', FloatTensorType([1, 1])),
                ('cardistancevalue', FloatTensorType([1, 1])),
                ('tirerating', StringTensorType([1, 1])),
                ('electricalrating', StringTensorType([1, 1])),
                ('fueltype', StringTensorType([1, 1])),
                ('interiorcolor', StringTensorType([1, 1]))
                ]


# Convert the KMeans model to ONNX format
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("kmeans_model6.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


In [19]:
kmeans

KMeans(random_state=42)

<373032x10397 sparse matrix of type '<class 'numpy.float64'>'
	with 23874048 stored elements in Compressed Sparse Row format>

In [10]:
len(data.columns)

86

Building the Recommender System

In [11]:
data.columns

Index(['requestedprice', 'orderid', 'islease', 'isfromleasecustomer',
       'vendortype', 'totalcarcost', 'autotelexvalue', 'datesalecancelled',
       'ismanuallychecked', 'ismetallic', 'tirerating', 'hasairco',
       'electricalrating', 'fueltype', 'interiorcolor', 'hascosts',
       'cardistancevalue', 'cartypename', 'hasautomatictransmission',
       'ishighrisk', 'catalogvalue', 'interiorrating', 'modelname',
       'enginekilowatt', 'vatmargin', 'maintenancevalue', 'enginehorsepower',
       'chassisnb', 'rdwdescription', 'carweight', 'maintenancedate',
       'bpmvalue', 'mechanicalrating', 'duedateapk', 'modelnameshort',
       'bodytype', 'enginedisplacement', 'exteriorrating',
       'expecteddistancevalue', 'xrayvalue', 'display_carphotos',
       'datevehicleregistration', 'createdon', 'remainingbpmvalue',
       'equipmentvalue', 'isrental', 'dateascription', 'exteriorcolor',
       'calculationnumber', 'extrapackagedescription', 'vehicletype',
       'xraynettvalue', 'c

In [12]:
[i for i in data.columns if 'car' in i]

['totalcarcost',
 'cardistancevalue',
 'cartypename',
 'carweight',
 'display_carphotos',
 'cartypeextention',
 'display_carphotosisdamage',
 'ismovingcar',
 'carstatus',
 'isimportcar',
 'isxbsdirectcar']

In [13]:
data['xbs$companyid']

KeyError: 'xbs$companyid'

In [ ]:
from collections import defaultdict

# Create a dictionary to store the clusters each user has bid on
user_clusters = defaultdict(list)

for _, row in data.iterrows():
    user_clusters[row['xbs$companyid']].append(data.loc[data['orderId'] == row['orderId'], 'cluster'].values[0])

# Recommend cars from similar clusters
def recommend_cars(user_id, num_recommendations=5):
    user_cluster = user_clusters[user_id]
    # Get cars from the same clusters
    recommended_cars = data[data['cluster'].isin(user_cluster)].sample(num_recommendations)
    return recommended_cars

# Example recommendation for a user
user_id = 1
recommend_cars(user_id)


KeyError: 'orderId'

Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Implement the recommender system on the training set
# (same steps as above, but using train_data instead of the entire dataset)

# Generate recommendations for users in the test set and evaluate
# (Assume we have a function `evaluate_recommendations` to calculate the metrics)

# Evaluate the recommender system
def evaluate_recommendations(test_data):
    y_true = []
    y_pred = []
    
    for _, row in test_data.iterrows():
        user_id = row['xbs$companyid']
        true_car_id = row['orderId']
        
        recommended_cars = recommend_cars(user_id)
        recommended_car_ids = recommended_cars['car_id'].values
        
        y_true.append(true_car_id)
        y_pred.append(true_car_id in recommended_car_ids)
    
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    
    return precision, recall, f1

precision, recall, f1 = evaluate_recommendations(test_data)
print(f'Precision: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}')
